# Introduction to AtomMan: System basics

This Notebook outlines the basics of using atomman to generate atomic systems in Python.

In [1]:
#Import atomman library
import atomman as am
import numpy as np

## 1. Generate a System

A System can be generated from scratch by combining an Atoms object and a Box object.

### 1.1 Create an Atoms object

The Atoms class contains the per-atom properties for all of the atoms. By default, each Atoms instance has two per-atom properties: atype and pos.  atype is an integer atomic type, and pos is the 3D vector position.  Other properties can be freely assigned to all atoms at any time with any name. More information on the Atoms class can be found in the [atomman.Atoms Notebook](in_depth_Notebooks/atomman.Atoms.ipynb).

In [2]:
#Create an Atoms object consistent with a simple face-centered cell and all atoms of the same type
atype = [1,1,1,1]
pos = [[0.0, 0.0, 0.0], [0.5, 0.5, 0.0], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5]]

#Assign values to atype and pos properties during creation.
atoms = am.Atoms(natoms=4, prop={'atype':atype, 'pos':pos})
print atoms

     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.500 |   0.500 |   0.000
      2 |       1 |   0.500 |   0.000 |   0.500
      3 |       1 |   0.000 |   0.500 |   0.500


### 1.2 Create a Box object

The Box class represents a generic parallelopid in space as four vectors: the vectors for the three independent box directions, and an origin position vector.  It can be created using lattice parameters, LAMMPS box terms, or by explicitly giving the underlying vectors. More information on the Box class can be found in the [atomman.Box Notebook](in_depth_Notebooks/atomman.Box.ipynb).

In [3]:
#Create a cubic box
box = am.Box(a=4.05, b=4.05, c=4.05)
print box

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]


### 1.3 Create a System by combining Atoms and Box

A System is then constructed by combining the Atoms and Box.  The term pbc is a tuple of three Boolean values indicating which directions are periodic (True). The argument "scale=True" scales the Atoms' pos values according to the Box definition. More information on the System class can be found in the [atomman.System Notebook](in_depth_Notebooks/atomman.System.ipynb).

In [4]:
system = am.System(atoms=atoms, box=box, pbc=(True, True, True), scale=True)
print system

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


### 1.4. Load a System from a file

Systems can also be generated by loading from a file.  Atomman supports loading from CIF, POSCAR, LAMMPS data, LAMMPS dump, and json/xml data models. More information on the different load functions and methods can be found in the [atomman.load Notebook](in_depth_Notebooks/atomman.load.ipynb).

In [5]:
#Create a LAMMPS dump file for 
with open('fcc.dump', 'w') as f:
    f.write("""
ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
4
ITEM: BOX BOUNDS pp pp pp
0.000000 4.050000
0.000000 4.050000
0.000000 4.050000
ITEM: ATOMS id type x y z
1 1 0.0000000000000e+00 0.0000000000000e+00 0.0000000000000e+00
2 1 2.0250000000000e+00 2.0250000000000e+00 0.0000000000000e+00
3 1 2.0250000000000e+00 0.0000000000000e+00 2.0250000000000e+00
4 1 0.0000000000000e+00 2.0250000000000e+00 2.0250000000000e+00""")

In [6]:
#Load from fcc.dump and show the system info
new_system, symbols = am.load('atom_dump', 'fcc.dump')
print new_system

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


## 2. System Manipulations

There are a number of built-in functions for manipulating the systems.

### 2.1 Rotating a System

In [7]:
#Define axes for rotation
axes = np.array([[1,1,0],[-1,1,0], [0,0,1]])

Generic rotation of a System can be performed using atomman.tools.rotate(). This function transforms the box axes while keeping the relative positions of the atoms.

In [8]:
print am.tools.rotate(system, axes)

avect =  [ 2.864, -2.864,  0.000]
bvect =  [ 2.864,  2.864,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.864 |   0.000 |   0.000
      2 |       1 |   1.432 |  -1.432 |   2.025
      3 |       1 |   1.432 |   1.432 |   2.025


For cubic systems, the atomman.tools.rotate_cubic() function can also be used.  It takes the axes information as crystallographic directions, and after rotating expands the cell such that all three directions remain periodic.

In [9]:
print am.tools.rotate_cubic(system, axes)

avect =  [ 5.728,  0.000,  0.000]
bvect =  [ 0.000,  5.728,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 8
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   2.864 |   0.000
      1 |       1 |   0.000 |   0.000 |   0.000
      2 |       1 |   2.864 |   0.000 |   0.000
      3 |       1 |   1.432 |   1.432 |   2.025
      4 |       1 |   1.432 |   4.296 |   2.025
      5 |       1 |   2.864 |   2.864 |   0.000
      6 |       1 |   4.296 |   1.432 |   2.025
      7 |       1 |   4.296 |   4.296 |   2.025


### 2.2. Multiplying the size of a System

A larger System (i.e. supercell) can be generated using the am.tools.supersize() function, or the System.supersize() method. 

- __am.tools.supersize(system, a_mult, b_mult, c_mult)__ returns a new system by multiplying the size of the given system by the a_mult, b_mult, and c_mult terms.

- __System.supersize(a_mult, b_mult, c_mult)__ alters the System it is called from by multiplying the size by the a_mult, b_mult, and c_mult terms.

Each of the \*\_mult terms can either be an integer or a tuple of integers. For single integers, the magnitude indicates how many times the system is multiplied in that direction, and the sign the direction in which the multiplication occurs.  Tuple terms allow for both negative and positive multipliers to be given at the same time.  Within the tuples, the first term must be less than or equal to zero, and the second term greater than or equal to zero, and the number of times the system is multiplied by in that direction is the difference between the second and first tuple terms.  

In [10]:
print system

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


In [11]:
#atomman.tools.supersize returns a new system
#a_mult = 1 does not change a direction.
#b_mult = -1 shifts system such that they are lower than the original origin
#c_mult = (-1, 1) multiplies the system by 2, centered around original origin
print am.tools.supersize(system, 1, -1, (-1, 1))

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  8.100]
origin = [ 0.000, -4.050, -4.050]
natoms = 8
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |  -4.050 |  -4.050
      1 |       1 |   2.025 |  -2.025 |  -4.050
      2 |       1 |   2.025 |  -4.050 |  -2.025
      3 |       1 |   0.000 |  -2.025 |  -2.025
      4 |       1 |   0.000 |  -4.050 |   0.000
      5 |       1 |   2.025 |  -2.025 |   0.000
      6 |       1 |   2.025 |  -4.050 |   2.025
      7 |       1 |   0.000 |  -2.025 |   2.025


In [12]:
#System.supersize() changes the system it was called on
system.supersize(2, 2, 2)
print system

avect =  [ 8.100,  0.000,  0.000]
bvect =  [ 0.000,  8.100,  0.000]
cvect =  [ 0.000,  0.000,  8.100]
origin = [ 0.000,  0.000,  0.000]
natoms = 32
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025
      4 |       1 |   4.050 |   0.000 |   0.000
      5 |       1 |   6.075 |   2.025 |   0.000
      6 |       1 |   6.075 |   0.000 |   2.025
      7 |       1 |   4.050 |   2.025 |   2.025
      8 |       1 |   0.000 |   4.050 |   0.000
      9 |       1 |   2.025 |   6.075 |   0.000
     10 |       1 |   2.025 |   4.050 |   2.025
     11 |       1 |   0.000 |   6.075 |   2.025
     12 |       1 |   4.050 |   4.050 |   0.000
     13 |       1 |   6.075 |   6.075 |   0.000
     14 |       1 |   6.075 |   4.050 |   2.025
     15 |       1 |   4.050 |   6.075 |   2.025
     16 |       1 |   0.

In [13]:
#Repeated calling of System.supersize() will cumulate changes
system.supersize(2, 2, 2)
print system

avect =  [16.200,  0.000,  0.000]
bvect =  [ 0.000, 16.200,  0.000]
cvect =  [ 0.000,  0.000, 16.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 256
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025
      4 |       1 |   4.050 |   0.000 |   0.000
      5 |       1 |   6.075 |   2.025 |   0.000
      6 |       1 |   6.075 |   0.000 |   2.025
      7 |       1 |   4.050 |   2.025 |   2.025
      8 |       1 |   0.000 |   4.050 |   0.000
      9 |       1 |   2.025 |   6.075 |   0.000
     10 |       1 |   2.025 |   4.050 |   2.025
     11 |       1 |   0.000 |   6.075 |   2.025
     12 |       1 |   4.050 |   4.050 |   0.000
     13 |       1 |   6.075 |   6.075 |   0.000
     14 |       1 |   6.075 |   4.050 |   2.025
     15 |       1 |   4.050 |   6.075 |   2.025
     16 |       1 |   0

### 2.3 Other manipulations

Tools also exist for transforming rhombohedral systems to octahedral systems, and transforming octahedral systems to orthorhombic systems. 

Further manipulations can be found in the atomman.defects submodule for adding more complicated modifications, such as allowing for point defects and dislocation monopoles to be created.

## 3. Basic Analysis Tools

### 3.1 Calculating position distance vectors

#### 3.1.1 System.dvect(pos_0, pos_1)

The dvect method gives the shortest vector distance between two points within the System by taking periodic boundaries into consideration. There are a number of options for the pos terms:

1. Single integer values will use the position of the atom in the system with that atom index.
2. Lists of integers will use the positions of all of the atoms with those atom indecies.
3. A 3D vector of floats will be used as the position.
4. A list of 3D vector floats will be used as the positions.

This function uses numpy vectorization meaning that it is considerably faster to supply lists for the terms as opposed to calling the function multiple times with single values.  The two pos terms don't have to be of the same type, but their lengths have to be compatible (either the same length, or one is singular).    

In [14]:
#Print the vector between atoms with indecies 1 and 5
print system.dvect(1,5)

[ 4.05  0.    0.  ]


In [15]:
#Print the vectors between all atoms and position [0.1, 0.1, 0.1]
pos = np.array([0.1, 0.1, 0.1])
a_ids = range(system.natoms)

print system.dvect(pos, a_ids)

[[-0.1   -0.1   -0.1  ]
 [ 1.925  1.925 -0.1  ]
 [ 1.925 -0.1    1.925]
 [-0.1    1.925  1.925]
 [ 3.95  -0.1   -0.1  ]
 [ 5.975  1.925 -0.1  ]
 [ 5.975 -0.1    1.925]
 [ 3.95   1.925  1.925]
 [-0.1    3.95  -0.1  ]
 [ 1.925  5.975 -0.1  ]
 [ 1.925  3.95   1.925]
 [-0.1    5.975  1.925]
 [ 3.95   3.95  -0.1  ]
 [ 5.975  5.975 -0.1  ]
 [ 5.975  3.95   1.925]
 [ 3.95   5.975  1.925]
 [-0.1   -0.1    3.95 ]
 [ 1.925  1.925  3.95 ]
 [ 1.925 -0.1    5.975]
 [-0.1    1.925  5.975]
 [ 3.95  -0.1    3.95 ]
 [ 5.975  1.925  3.95 ]
 [ 5.975 -0.1    5.975]
 [ 3.95   1.925  5.975]
 [-0.1    3.95   3.95 ]
 [ 1.925  5.975  3.95 ]
 [ 1.925  3.95   5.975]
 [-0.1    5.975  5.975]
 [ 3.95   3.95   3.95 ]
 [ 5.975  5.975  3.95 ]
 [ 5.975  3.95   5.975]
 [ 3.95   5.975  5.975]
 [ 8.    -0.1   -0.1  ]
 [-6.175  1.925 -0.1  ]
 [-6.175 -0.1    1.925]
 [ 8.     1.925  1.925]
 [-4.15  -0.1   -0.1  ]
 [-2.125  1.925 -0.1  ]
 [-2.125 -0.1    1.925]
 [-4.15   1.925  1.925]
 [ 8.     3.95  -0.1  ]
 [-6.175  5.975 

#### 3.1.2 atomman.tools.dvect(pos_0, pos_1, box, pbc)

This function is similar to the System.dvect() method, but it is independent of any System. As the System information is not built-in, box and pbc information must be given, and indexes cannot be used for pos_0 and pos_1. 

#### 3.1.3 atomman.tools.displacement(system_0, system_1)

This returns the displacement of all atoms in system_1 relative to a base configuration in system_0.  Essentially, it sets up and calls atomman.tools.dvect() using system_1's boundary information.

### 3.2 Neighbor lists

#### 3.2.1 atomman.tools.nlist(system, cutoff, cmult=1)

The nlist method computes neighbor lists for all of the atoms in the system within the specified cutoff. The optional term cmult alters how atoms are binned during the calculation, with the default seeming to be the fastest.

Returns a numpy array in which the coordination number and list of neighbor atoms is given for each atom.

In [16]:
nlist = am.tools.nlist(system, cutoff=3)

print 'Average coordination =',
print np.mean(nlist[:,0])

for ninfo in nlist:
    print ninfo[1:1+ninfo[0]]


Average coordination = 12.0
[  1   2   3  37  38  73  75 109 146 147 182 219]
[  0   2   3   4   7   8  10  12 146 147 151 154]
[ 0  1  3  4  7 16 17 20 73 75 79 89]
[ 0  1  2  8 10 16 17 24 37 38 46 53]
[  1   2   5   6   7  73  77  79 146 150 151 223]
[  4   6   7  12  14  32  35  40 150 151 158 179]
[  4   5   7  20  21  32  35  48  77  79  93 107]
[ 1  2  4  5  6 10 12 14 17 20 21 28]
[  1   3   9  10  11  37  45  46 147 154 155 190]
[  8  10  11  12  15  64  66  68 154 155 159 210]
[ 1  3  7  8  9 11 12 15 17 24 25 28]
[  8   9  10  24  25  45  46  61  64  66  80 102]
[  1   5   7   9  10  13  14  15 151 154 158 159]
[ 12  14  15  40  43  68  70  96 158 159 187 214]
[ 5  7 12 13 15 21 28 29 35 40 43 56]
[ 9 10 12 13 14 25 28 29 66 68 70 84]
[  2   3  17  18  19  38  53  54  75  89  91 125]
[ 2  3  7 10 16 18 19 20 23 24 26 28]
[ 16  17  19  20  23  89  91  95 128 129 132 201]
[ 16  17  18  24  26  53  54  62 128 129 136 165]
[ 2  6  7 17 18 21 22 23 79 89 93 95]
[ 6  7 14 20 22 23

#### 3.2.2 System.nlist(cutoff, cmult=1)

This is identical to calling atomman.tools.nlist(), except that the computed neighbor list is saved as a System property instead of returned.

In [17]:
system.nlist(3)
print np.all(system.prop['nlist'] == nlist)

True


#### 4. File cleanup

In [18]:
import os
os.remove('fcc.dump')
os.remove('fcc.dump.json')